# PyTorch 中的循环神经网络模块

### 1.`RNN()`:
- input_size 表示输入 $x_t$ 的特征维度

- hidden_size 表示输出的特征维度

- num_layers 表示网络的层数

- nonlinearity 表示选用的非线性激活函数，默认是 'tanh'

- bias 表示是否使用偏置，默认使用

- batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位

- dropout 表示是否在输出层应用 dropout

- bidirectional 表示是否使用双向的 rnn，默认是 False

### 2.`RNNCell()`:
- input_size
- hidden_size
- bias
- nonlinearity

In [1]:
import torch
from torch.autograd import Variable
from torch import nn

### 1.使用nn.RNNCell()

In [2]:
#定义一个单步的rnn
rnn_single=nn.RNNCell(input_size=100,hidden_size=200)

In [8]:
#访问其中的参数
a=rnn_single.weight_hh
print(type(a))
print(a.shape)
print(a)

<class 'torch.nn.parameter.Parameter'>
torch.Size([200, 200])
Parameter containing:
tensor([[ 0.0623, -0.0017, -0.0243,  ...,  0.0045, -0.0348, -0.0245],
        [-0.0536,  0.0099, -0.0676,  ...,  0.0231,  0.0624,  0.0654],
        [ 0.0546,  0.0631,  0.0382,  ..., -0.0012,  0.0456,  0.0353],
        ...,
        [-0.0089, -0.0003,  0.0315,  ...,  0.0249, -0.0190, -0.0662],
        [ 0.0426, -0.0568, -0.0666,  ...,  0.0310, -0.0290, -0.0634],
        [ 0.0368, -0.0443,  0.0068,  ...,  0.0241,  0.0451,  0.0433]],
       requires_grad=True)


In [10]:
# 构造一个序列，长为 6，batch 是 5， 特征是 100
x=Variable(torch.randn(6,5,100))#sequence=6,batch=5,input_size=100

In [11]:
# 定义初始的记忆状态
h_t=Variable(torch.zeros(5,200))#batch=5,hidden_size=200

In [12]:
#传入rnn,模拟了rnn的迭代过程
out=[]
for i in range(6):
    h_t=rnn_single(x[i],h_t)#把输入tensor和初始启动记忆tensor
    out.append(h_t)

In [14]:
#输出最后的state tensor
print(h_t.shape)

torch.Size([5, 200])


In [15]:
#看一下out
len(out)

6

In [16]:
out[0].shape

torch.Size([5, 200])

### 2.使用nn.RNN()

In [17]:
rnn_seq=nn.RNN(100,200)

In [20]:
#访问其中的参数
a=rnn_seq.weight_hh_l0
print(a)
print(a.shape)

Parameter containing:
tensor([[-0.0449,  0.0621,  0.0357,  ...,  0.0145,  0.0256, -0.0150],
        [ 0.0219,  0.0363, -0.0539,  ...,  0.0149,  0.0189, -0.0651],
        [-0.0138, -0.0648, -0.0410,  ..., -0.0561,  0.0216,  0.0004],
        ...,
        [ 0.0328, -0.0165,  0.0406,  ...,  0.0408, -0.0678, -0.0704],
        [ 0.0016,  0.0201, -0.0578,  ..., -0.0404, -0.0566, -0.0480],
        [ 0.0681, -0.0066, -0.0701,  ...,  0.0458,  0.0673,  0.0132]],
       requires_grad=True)
torch.Size([200, 200])


In [21]:
out, h_t = rnn_seq(x) # 使用默认的全 0 隐藏状态

In [24]:
a=h_t
print(type(a))
print(a.shape)
print(type(out))
print(out.shape)

<class 'torch.Tensor'>
torch.Size([1, 5, 200])
<class 'torch.Tensor'>
torch.Size([6, 5, 200])


In [26]:
# 自己定义初始的隐藏状态
h_0 = Variable(torch.randn(1, 5,
                           200)) # 这里的隐藏状态的大小有三个维度，分别是 (num_layers * num_direction, batch, hidden_size)
out, h_t = rnn_seq(x, h_0)

In [28]:
a=h_t
print(a.shape)

torch.Size([1, 5, 200])


In [29]:
print(out.shape)

torch.Size([6, 5, 200])


同时输出的结果也是 (seq, batch, feature)

一般情况下我们都是用 **nn.RNN()** 而不是 nn.RNNCell()，因为 nn.RNN() 能够避免我们手动写循环，非常方便，同时如果不特别说明，我们也会选择使用默认的全 0 初始化隐藏状态

## 二. LSTM
LSTM 和基本的 RNN 是一样的，他的参数也是相同的，同时他也有 nn.LSTMCell() 和 nn.LSTM() 两种形式，跟前面讲的都是相同的，我们就不再赘述了，下面直接举个小例子
### 1.nn.LSTM(input_size, output_size, num_layers)

In [39]:
lstm_seq=nn.LSTM(50,100,num_layers=2) # 输入维度 100，输出 200，两层

In [40]:
a=lstm_seq.weight_hh_l0 #第一层的h_t权重
print(a.shape)

torch.Size([400, 100])


In [41]:
lstm_input=Variable(torch.randn(10,3,50))#序列10, batch=3,输入50
out,(h,c)=lstm_seq(lstm_input)

注意这里 LSTM 输出的隐藏状态有两个，h 和 c，就是上图中的每个 cell 之间的两个箭头，这两个隐藏状态的大小都是相同的，(num_layers * direction, batch, feature)

In [42]:
h.shape # 两层, Batch是3,特征是100

torch.Size([2, 3, 100])

In [43]:
c.shape

torch.Size([2, 3, 100])

In [44]:
out.shape

torch.Size([10, 3, 100])

我们可以不使用默认的隐藏状态，这是需要传入两个张量

In [45]:
h_init=Variable(torch.randn(2,3,100))#shape的定义是[num_layers,batch,output_size]
c_init=Variable(torch.randn(2,3,100))

In [46]:
out,(h,c)=lstm_seq(lstm_input,(h_init,c_init))#在MT中seq2seq中,这句话是我们需要用到的.

In [47]:
h.shape

torch.Size([2, 3, 100])

In [48]:
c.shape

torch.Size([2, 3, 100])

In [49]:
out.shape

torch.Size([10, 3, 100])

### 2.GRU

In [50]:
gru_seq=nn.GRU(10,20)
gru_input=Variable(torch.randn(3,32,10))
out,h=gru_seq(gru_input)

In [52]:
a=gru_seq.weight_hh_l0
print(a)
print(a.shape)

Parameter containing:
tensor([[ 0.0858,  0.1167, -0.2150,  ...,  0.1816,  0.0614, -0.0686],
        [ 0.0204,  0.1537,  0.1798,  ..., -0.1466,  0.0764, -0.2056],
        [ 0.1458, -0.0388,  0.0597,  ..., -0.1942, -0.0172, -0.0745],
        ...,
        [ 0.2145, -0.0718,  0.1385,  ..., -0.1296,  0.2046,  0.0909],
        [ 0.2044,  0.2040, -0.0603,  ..., -0.1046, -0.2156, -0.1384],
        [ 0.1613,  0.1350, -0.0216,  ...,  0.1015,  0.0816, -0.0669]],
       requires_grad=True)
torch.Size([60, 20])


In [53]:
h.shape

torch.Size([1, 32, 20])

In [54]:
out.shape

torch.Size([3, 32, 20])

## 小结:
- nn.LSTM(input_size,output_size,num_layers)
- nn.GRU(input_size, output_size,num_layers)